# Automatic Differentiation

Recall from Section 2.4 that calculating derivatives is the crucial step in all of the optimization algorithms that we will use to train deep networks. While the calculations are straightforward, working them out by hand can be tedious and error-prone, and this problem only grows as our models become more complex.

Fortunately all modern deep learning frameworks take this work off of our plates by offering automatic differentiation (often shortened to autograd). As we pass data through each successive function, the framework builds a computational graph that tracks how each value depends on others. To calculate derivatives, automatic differentiation works backwards through this graph applying the chain rule. The computational algorithm for applying the chain rule in this fashion is called backpropagation.

While autograd libraries have become a hot concern over the past decade, they have a long history. In fact the earliest references to autograd date back over half of a century (Wengert, 1964). The core ideas behind modern backpropagation date to a PhD thesis from 1980 (Speelpenning, 1980) and were further developed in the late 1980s (Griewank, 1989). While backpropagation has become the default method for computing gradients, it is not the only option. For instance, the Julia programming language employs forward propagation (Revels et al., 2016). Before exploring methods, let’s first master the [Zygote](https://fluxml.ai/Zygote.jl/) package.

## A Simple Function

To start, we assign x an initial value.

In [1]:
x = collect(1.0:4.0)

4-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0

We now define a function of x.

In [2]:
using LinearAlgebra

y(x) = 2x⋅x

y (generic function with 1 method)

We can now take the gradient of y with respect to x by calling `gradient` method.

In [3]:
using Zygote

g = gradient(y,x)

([4.0, 8.0, 12.0, 16.0],)

We can now verify that the automatic gradient computation and the expected result are identical.

In [4]:
first(g) == x.*4

true

Now let’s calculate another function of x and take its gradient.

In [5]:
y(x) = sum(x)
gradient(y,x)

(Fill(1.0, 4),)

## Backward for Non-Scalar Variables

When y is a vector, the most natural interpretation of the derivative of y with respect to a vector x is a matrix called the Jacobian that contains the partial derivatives of each component of y with respect to each component of x. Likewise, for higher-order y and x, the differentiation result could be an even higher-order tensor.

While Jacobians do show up in some advanced machine learning techniques, more commonly we want to sum up the gradients of each component of y with respect to the full vector x, yielding a vector of the same shape as x. For example, we often have a vector representing the value of our loss function calculated separately for each example among a batch of training examples. Here, we just want to sum up the gradients computed individually for each example.

In [6]:
y(x) = x.*x
gradient(x->sum(y(x)),x)

([2.0, 4.0, 6.0, 8.0],)